In [1]:
from tqdm.notebook import tqdm
import wandb

from WindModel import *

wandb.login()

wandb: Currently logged in as: bhavye-mathur. Use `wandb login --relogin` to force relogin


True

In [2]:
WindDataset.init(0.1)

train = WindDataset("train")
validation = WindDataset("validation")
test = WindDataset("test")

del WindDataset.data

In [3]:
OUTPUT_SIZE = 1
INPUT_SIZE = 13

LOSS_FUNC = torch.nn.MSELoss

In [4]:
def get_dense_model(input_size: int,
                    hidden_sizes: list[int],
                    output_size: int,
                    activation_func: callable):
    layers = []

    for size in hidden_sizes:
        layers.append(torch.nn.Linear(input_size, size))
        layers.append(activation_func())
        input_size = size

    layers.append(torch.nn.Linear(input_size, output_size))

    return torch.nn.Sequential(*layers)

In [5]:
class SinusoidalLayer(torch.nn.Module):
    def __init__(self, size_in: int, size_out: int, device=None, dtype=None):
        factory_kwargs = {'device': DEVICE, 'dtype': dtype}

        super().__init__()
        self.size_in = size_in
        self.size_out = size_out

        self.periods = torch.linspace(0.01, 100, size_out, **factory_kwargs).repeat((size_in, 1))

        self.weights = torch.nn.Parameter(torch.empty((size_out, size_out), **factory_kwargs))
        self.phase = torch.nn.Parameter(torch.zeros(size_out, **factory_kwargs))
        # self.bias = torch.nn.Parameter(torch.empty(size_out, **factory_kwargs))

        torch.nn.init.kaiming_uniform_(self.weights, a=2.236)
        fan_in, _ = torch.nn.init._calculate_fan_in_and_fan_out(self.weights)
        bound = 1 / (fan_in ** 0.5)
        # torch.nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, x):
        out = torch.mm(x, self.periods)
        out = torch.add(out, self.phase)
        out = torch.sin(out)
        out = torch.mm(out, self.weights)
        # out = torch.add(out, self.bias)
        return out


def get_sinusoidal_model(input_size: int,
                         periods: int,
                         hidden_sizes: list[int],
                         output_size: int,
                         activation_func: callable):
    layers = [SinusoidalLayer(input_size, periods),
              activation_func()]
    input_size = periods

    for size in hidden_sizes:
        layers.append(torch.nn.Linear(input_size, size))
        layers.append(activation_func())
        input_size = size

    layers.append(torch.nn.Linear(input_size, output_size))

    return torch.nn.Sequential(*layers)


In [8]:
def evaluate_one_epoch(model, epoch):
    mse = 0
    mae = 0

    with torch.no_grad():
        prediction = model(validation.x).squeeze()

        mse += torch.nn.functional.mse_loss(prediction, validation.y)
        mae += torch.nn.functional.l1_loss(prediction, validation.y)

    wandb.log({"val_rmse": (mse ** 0.5) * stds[VARIABLE],
               "val_mae": mae * stds[VARIABLE]})


def train_one_batch(model, optimizer, criterion, batch, batch_idx):
    optimizer.zero_grad()

    inputs, targets = batch

    prediction = model(inputs).squeeze()
    loss = criterion(prediction, targets)

    loss.backward()
    optimizer.step()

    if batch_idx != 0 and batch_idx % 100 == 0:
        with torch.no_grad():
            rmse = (torch.nn.functional.mse_loss(prediction, targets) ** 0.5) * stds[VARIABLE]
            wandb.log({"train_loss": loss,
                       "train_rmse": rmse})


def train_one_epoch(model, optimizer, criterion, epoch, batch_size):
    n = len(train)

    for i in range(len(train) // batch_size):
        lower_i = i * batch_size
        upper_i = min((i + 1) * batch_size, n)

        batch_x = train.x[lower_i: upper_i]
        batch_y = train.y[lower_i: upper_i]

        train_one_batch(model, optimizer, criterion, (batch_x, batch_y), i)


def main():
    # wandb.init()

    learning_rate = wandb.config.learning_rate
    batch_size = wandb.config.batch_size
    layers = wandb.config.layers
    periods = wandb.config.periods
    epochs = wandb.config.epochs
    activation = wandb.config.activation

    activation = getattr(torch.nn, activation)

    model = get_dense_model(INPUT_SIZE, layers, OUTPUT_SIZE, activation)
    model = model.to(DEVICE)
    print(model)

    criterion = LOSS_FUNC()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    if (scheduler := wandb.config.lr_scheduler) is None:
        scheduler = None
    elif scheduler == "StepLR":
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, **wandb.config.lr_scheduler_kwargs)

    # wandb.watch(model, log_freq=100)

    for ep in tqdm(range(epochs)):
        print(ep, end=" ")

        wandb.log({"epoch": ep})

        model.train()
        train_one_epoch(model, optimizer, criterion, ep, batch_size)

        model.eval()
        evaluate_one_epoch(model, ep)

        if scheduler:
            scheduler.step()
            wandb.log({"lr": scheduler.get_last_lr()[-1]})

In [9]:
config = {
    "batch_size": 65536,
    "learning_rate": 0.0005,
    "lr_scheduler": "StepLR",
    "lr_scheduler_kwargs": {"step_size": 16, "gamma": 0.3},
    "layers": [1024, 512],
    "activation": "LeakyReLU",
    "estimate_quantile": ESTIMATE_QUANTILE,
    "dataset": DATASET,
    "epochs": 40,
    "periods": 1024,
}

wandb.init(project=f"MERRA2-{VARIABLE}-July2023", dir="wandb-local", config=config)

main()
wandb.finish()

epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▅▃▂▂▂▂▁▁▁▁▁▁
train_rmse,█▅▄▃▂▂▂▂▂▁▁▁▁
val_mae,█▆▄▃▂▂▂▂▁▁▁▁▁
val_rmse,█▅▄▃▂▂▂▂▂▁▁▁▁
epoch,13
lr,0.0005
train_loss,0.26701
train_rmse,5.08044
val_mae,3.62003


Sequential(
  (0): Linear(in_features=13, out_features=1024, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=1024, out_features=512, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=512, out_features=1, bias=True)
)


  0%|          | 0/40 [00:00<?, ?it/s]

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 

KeyboardInterrupt: 

In [ ]:
sweep_configuration = {
    "method": "bayes",
    "name": f"sweep-{DATASET}-Sinusoidal",
    "metric": {
        "goal": "minimize",
        "name": "val_rmse"
    },
    "parameters": {
        "batch_size": {"values": [8192, 16384, 32768, 65536]},
        "periods": {"values": [32, 64, 128, 256, 512, 1024]},
        "learning_rate": {"max": 0.001, "min": 0.00001},
        "lr_scheduler": {"values": [None, "StepLR"]},
        "lr_scheduler_kwargs": {"parameters": {"step_size": {"max": 20, "min": 10},
                                               "gamma": {"max": 0.85, "min": 0.15}}},
        "layers": {"values": [(256, 128), (512, 256), (1024, 512),
                              (1024, 512, 256), (256, 256, 32), (512, 256, 128), (128, 64, 32)]},
        "epochs": {"value": 40},
        "activation": {"values": ["ReLU", "PReLU", "LeakyReLU", "ELU", "Softplus"]},
        "estimate_quantile": {"value": ESTIMATE_QUANTILE},
        "dataset": {"value": DATASET},
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 3,
        "eta": 2
    }
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project=f"MERRA2-{VARIABLE}-July2023")
wandb.agent(sweep_id, function=main)

In [ ]:
def test(model, dl):
    model.eval()

    mse = 0
    mae = 0

    with torch.no_grad():
        for inputs, targets in tqdm(dl):
            prediction = model(inputs).squeeze()

            mse += torch.nn.functional.mse_loss(prediction, targets)
            mae += torch.nn.functional.l1_loss(prediction, targets)

    return (mse / len(dl)) ** 0.5 * stds[VARIABLE], (mae / len(dl)) * stds[VARIABLE]


test_dl = DataLoader(test, batch_size=2048, shuffle=False)
test_rmse, test_mae = test(model, test)

print(f"RMSE: {test_rmse} m/s")
print(f"MAE:  {test_mae} m/s")
